# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096917


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/31 [00:05<01:20,  2.78s/it]

Rendering models:  10%|▉         | 3/31 [00:05<00:56,  2.02s/it]

Rendering models:  13%|█▎        | 4/31 [00:05<00:39,  1.47s/it]

Rendering models:  16%|█▌        | 5/31 [00:08<00:42,  1.64s/it]

...

Rendering models:  77%|███████▋  | 24/31 [00:11<00:01,  5.51it/s]

Rendering models:  87%|████████▋ | 27/31 [00:11<00:00,  7.23it/s]

Rendering models:  94%|█████████▎| 29/31 [00:12<00:00,  6.10it/s]

Rendering models: 100%|██████████| 31/31 [00:12<00:00,  5.32it/s]

Jnursssmith5263                       0.001647
not-logged-in-673e227f920b24558cc1    0.000813
Kartoon1x                             0.000410
jnarayanbvg                           0.001156
not-logged-in-c6917d9b19f41e6c2995    0.003260
awright5                              0.003437
Quarkgc                               0.000719
framosuw                              0.001066
M4kc                                  0.001531
not-logged-in-f3082f2a86c2c9f95d30    0.005680
jmartinez468                          0.013584
kanghan                               0.003481
not-logged-in-6d3e4eb12f2149553110    0.001880
not-logged-in-0cfbc5dddba6fbc5ea1d    0.162923
pangeli5                              0.000532
not-logged-in-396e5180e43142af74c3    0.001607
w7250369                              0.004694
KiraHuter                             0.000829
atayl233                              0.000625
slambier                              0.000413
Jackson2802                           0.000535
sirtylerli   

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())